# Selección de modelos

## Optimización de hiperparámetros

## Autores

Angelica Agudelo (maagudeloro@unal.edu.co)

Pablo González (pgonzalezb@unal.edu.co)

Dana Reyes (dreyesmo@unal.edu.co)

## Referencias

Selección de modelos: https://scikit-learn.org/stable/model_selection.html#model-selection

Librería para búsqueda por optimización bayesiana: https://optuna.readthedocs.io/en/stable/

Librería para búsqueda de grilla: https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html

Librería para búsqueda aleatoria: https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.RandomizedSearchCV.html

Nota histórica:

- https://en.wikipedia.org/wiki/Random_search

- https://en.wikipedia.org/wiki/Random_optimization

- https://en.wikipedia.org/wiki/Bayesian_optimization

## Introducción

Los hiperparámetros son parámetros que no se aprenden directamente dentro de los estimadores. En general, se pasan como argumentos al constructor de las clases del estimador. Cualquier parámetro proporcionado al construir un estimador puede optimizarse con distintos métodos.

En scikit-learn se proporcionan dos enfoques genéricos para la búsqueda de parámetros: para valores dados, GridSearchCV considera exhaustivamente todas las combinaciones de parámetros, mientras que RandomizedSearchCV puede muestrear una cantidad determinada de candidatos de un espacio de parámetros con una distribución específica.

Por otro lado, el ajuste de hiperparámetros por medio del razonamiento bayesiano, o la optimización bayesiana, puede reducir el tiempo necesario para llegar al conjunto óptimo de parámetros y brindar un mejor rendimiento de generalización en el conjunto de prueba. Lo hace teniendo en cuenta la información sobre las combinaciones de hiperparámetros que ha visto hasta ahora al elegir el conjunto de hiperparámetros para evaluar a continuación. Al elegir sus combinaciones de parámetros de manera informada, se permite enfocarse en aquellas áreas del espacio de parámetros que cree que traerán los puntajes de validación más prometedores. Este enfoque generalmente requiere menos iteraciones para llegar al conjunto óptimo de valores de hiperparámetros.

A continuación hablaremos de las técnicas utilizadas para evaluar y seleccionar el mejor modelo según un conjunto de posibles valores de hiperparámetros. Revisaremos distintas técnicas de selección de hiperparámetros como por ejemplo una búsqueda aleatoria (random search), búsqueda de grilla (grid search) y optimización bayesiana para la optimización de hiperparámetros.

Cada sección de este capitulo incluirá un contexto histórico, las definiciones y ecuaciones matemáticas necesarias para entender la técnica desde el punto de vista científico, y una implementación en Python con datos extraídos de Kaggle.

## Nota histórica

Según información sacada de wikipedia. En 1953 la Revista de la Asociación Estadounidense de Estadística publicó un artículo con el nombre "Recent Advances in Finding Best Operating Conditions" ('Avances recientes en la búsqueda de las mejores condiciones de funcionamiento') escrito por el doctor R.L. Anderson donde revisó el progreso de los métodos usados para evaluar las condiciones experimentales en las reacciones químicas por varios científicos. El método en general consistía en encontrar el máximo o mínimo de problemas usando una serie de conjeturas distribuidas con un cierto patrón en el espacio de búsqueda, el patrón podía ser la búsqueda por grilla, una búsqueda secuencial en cada parámetro o una combinación de ambos; la búsqueda se aplica secuencialmente a cada parámetro y saca las mejores conjeturas de la última secuencia.

El término de búsqueda aleatoria se atribuye a Rastrigin, el cual, en una publicación en 1964 llamada "The convergence of the random search method in the extremal control of a many parameter system" ('La convergencia del método de búsqueda aleatoria en el control externo de un sistema con muchos parámetros') presenta un análisis matemático básico sobre la búsqueda aleatoria explicando que este método funciona moviéndose iterativamente a mejores posiciones en el espacio de búsqueda, estas posiciones se muestrean por una hiperesfera que rodea la posición actual. Posteriormente, en 1965 se menciona el término de optimización aleatoria en un artículo del mismo nombre realizado por Mátyás, en donde al igual que la búsqueda aleatoria se presenta un análisis matemático básico que explica como funciona este método que hace parte de los métodos de optimización numérica que no requieren que se optimice el gradiente; su funcionamiento es el mismo que en la búsqueda aleatoria, pero las posiciones se muestrean usando una distribución que puede ser normal.

Finalmente, Jonas Mockus publica una series de artículos sobre la optimización bayesiana entre las décadas de las 70s y 80s, y en consecuencia, el término de optimización bayesiana se le atribuye.

### Pasos previos (importar datos y ajustar modelo base)

Con el fin de realizar ejemplos con cada una de las técnicas distintas que vamos a exponer, utilizaremos un conjunto de datos muy simple sacado de Kaggle (https://www.kaggle.com/datasets/mirichoi0218/insurance) al cual le ajustaremos un modelo de Potenciación del gradiente (GradientBoosting) para regresión con los hiperparámetros por defecto por lo cual lo usaremos como modelo base, y a este modelo es al que le haremos la optimización de hiperparámetros.

Como métrica de rendimiento, utilizaremos el coeficiente de determinación (R2).

In [1]:
!pip install -r requirements.txt

You should consider upgrading via the '/root/venv/bin/python -m pip install --upgrade pip' command.


In [2]:
import pandas as pd

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import r2_score

In [3]:
data = pd.read_csv('https://raw.githubusercontent.com/pgonzalezb4/MineriaDeDatos-2022-2/main/data/insurance.csv')
data.head()

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520


In [4]:
data.dtypes

age           int64
sex          object
bmi         float64
children      int64
smoker       object
region       object
charges     float64
dtype: object

In [5]:
lb_encoder = LabelEncoder()
data.sex = lb_encoder.fit_transform(data.sex)
data.smoker = lb_encoder.fit_transform(data.smoker)
data.region = lb_encoder.fit_transform(data.region)

In [6]:
data.head()

,age,sex,bmi,children,smoker,region,charges
0,19,0,27.900,0,1,3,16884.92400
1,18,1,33.770,1,0,2,1725.55230
2,28,1,33.000,3,0,2,4449.46200
3,33,1,22.705,0,0,1,21984.47061
4,32,1,28.880,0,0,1,3866.85520


In [7]:
X = data.iloc[:, :-1].values
y = data.iloc[:, -1].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, shuffle=True, random_state=2)

reg = GradientBoostingRegressor() # modelo con hiperparametros por defecto
reg.fit(X_train, y_train)

y_pred = reg.predict(X_test)

r2 = r2_score(y_test, y_pred)
print('R2:', r2)

R2: 0.877581453601204


Ahora que ya tenemos un modelo base con un R2 de 0.87, procedemos a utilizar las distintas técnicas para la optimización de hiperparámetros y ver que resultado obtenemos.

## Búsqueda por grilla

### Fundamento matemático

Los hiperparámetros de un modelo son valores que deben especificarse de antemano y generalmente no se obtienen de los datos por lo que deben ser seleccionados por el científico de datos. Es de gran importancia hacer una correcta selección dichos parámetros para evitar inconvenientes en el ajuste del modelo. Pueden elegirse entonces parámetros de diferentes tamaños, incorporarlos al modelo y seleccionar los de mejor rendimiento. 

El método de búsqueda por grilla también conocido como búsqueda en cuadrícula o exhaustiva consiste en poner a prueba todas las posibles combinaciones de valores que se le proporcione en los parámetros. Esta prueba no se realiza de forma aleatoria sino en puntos del espacio repartidos en forma de cuadrícula. A partir de este algoritmo de optimización se seleccionan los mejores parámetros del espacio dado. Este método es conocido por su uso en aprendizaje de maquina para obtener los parámetros en los que el modelo ofrece la mejor precisión. Puede emplearse en diferentes problemas de optimización. Por ejemplo, la cantidad de árboles de decisión en el bosque aleatorio, la cantidad de nodos en cada capa de una red neuronal artificial o cantidad de capas ocultas etc.

En su implementación, el usuario lista los hiperparámetros y su producto cartesiano dado por permutaciones y combinaciones será su espacio. Así, cada dimensión será un hiperparámetro y cada punto representa la configuración al modelo. El algoritmo utiliza este espacio para entrenar el modelo y selecciona la combinación de parámetros con el error de validación más pequeño. Es decir que, sea $\Omega^*$ el espacio dado de hiperparámetros $V=(v_1, v_2, \ldots v_m)$. El método de grid seacrh define un vector de límites inferiores $a=(a_1, a_2, \ldots a_m)$ y superiores $b=(b_1, b_2, \ldots b_m)$ para cada componente del espacio $V$ y se toman $n$ puntos en cada intervalo $[a_i,b_i]$ con el mismo espacio entre cada punto. Esto dará como resultado $mn$ puntos de cuadrícula posibles para verificar en el modelo. Se calcula para cada uno y se eligen los hiperparámetros para los cuales el modelo tiene menor error.

Si se consideran dos hiperparámetros, se tendría en este caso un espacio con dos dimensiones en el que se construye una cuadricula con los puntos a verificar de la siguiente manera

<center><img src="GridSearch.png" width="300" height="300"></center>

Nota. Imagen por Angélica Agudelo

Se puede importar GridSearchCV que nos permitirá la implementación del método búsqueda de cuadrícula mediante sklearn como sigue

In [8]:
from sklearn.model_selection import GridSearchCV

### Ejemplos

Definimos los parámetros del modelo que se desean obtener mediante la búsqueda de cuadrícula, para este caso se van a definir 7 parámetros de la siguiente manera

In [9]:
from sklearn.model_selection import ParameterGrid
param_grid = {'loss': ['squared_error', 'absolute_error', 'huber','quantile'],
              'learning_rate': [0.05, 0.1, 0.15],
              'n_estimators': [50,100,150],
              'min_samples_split': [2, 3],
              'min_samples_leaf': [1, 2],
              'max_depth': [1, 2],
              'max_features': ['sqrt', 'log2']}

Una vez definido el espacio de los parámetros realizamos la búsqueda de aquellos que se ajustan mejor al modelo

In [10]:
from sklearn.ensemble import GradientBoostingRegressor
GBR = GradientBoostingRegressor() 
grid_GBR = GridSearchCV(GBR, param_grid, refit = True, verbose = 0,n_jobs=-1) 

In [11]:
X = data.iloc[:, :-1].values
Y = data.iloc[:, -1].values
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, shuffle=True, random_state=2)

grid_GBR.fit(X_train, y_train)

GridSearchCV(estimator=GradientBoostingRegressor(), n_jobs=-1,
             param_grid={'learning_rate': [0.05, 0.1, 0.15],
                         'loss': ['squared_error', 'absolute_error', 'huber',
                                  'quantile'],
                         'max_depth': [1, 2], 'max_features': ['sqrt', 'log2'],
                         'min_samples_leaf': [1, 2],
                         'min_samples_split': [2, 3],
                         'n_estimators': [50, 100, 150]})

Encontramos los parámetros y con estos podemos calcular el coeficiente de determinación del modelo construido con ellos

In [12]:
print(" Resulatdos Grid Search " )
print("\n Los mejores parámetros encontrados en la búsqueda son:\n",grid_GBR.best_params_)

y_pred= grid_GBR.predict(X_test)
r2=r2_score(y_test,y_pred)
print("\n El coeficiente de determinación del modelo es:\n",r2)

 Resulatdos Grid Search 

 Los mejores parámetros encontrados en la búsqueda son:
 {'learning_rate': 0.15, 'loss': 'huber', 'max_depth': 2, 'max_features': 'log2', 'min_samples_leaf': 2, 'min_samples_split': 3, 'n_estimators': 150}

 El coeficiente de determinación del modelo es:
 0.8650136879445166


Nótese que aunque se tomaron en cuenta los parámetros escogidos en la búsqueda de grilla el coeficiente de determinación disminuyó pues en la búsqueda no se están considerando todos los posibles valores de hiperparámetros por el costo computacional

## Búsqueda aleatoria

### Fundamento matemático

La búsqueda aleatoria (random search) hace parte de los métodos de optimización directa al no requerir de la optimización del gradiente. Este método como su nombre lo dice, hace una búsqueda aleatoria sobre los parámetros, donde se sacan muestras independientes a partir de una distribución uniforme sobre los posibles valores para los parámetros. Las muestras se toman sobre el mismo espacio de configuración que se abarcaría con la búsqueda por grilla.

<center><img src="RS.png" width="400"></center>


Nota. Adaptado de Randomized Search [Imagen], por Maël Fabien, 2020, Github (https://maelfabien.github.io/machinelearning/Explorium4/#what-is-hyperparameter-optimization).

Según wikipedia el nombre de "búsqueda aleatoria" se atribuye a Rastrigin quien hizo una presentación temprana del método junto con un algoritmo matemático básico. Este funciona moviéndose iterativamente a mejores posiciones en el espacio de búsqueda, que se muestrean desde una hiperesfera que rodea la posición actual. 

Es decir, sea $f:\mathbb{R}^n \rightarrow \mathbb{R}$ la función a ser minimizada y sea $x \in \mathbb{R}^n$ un valor candidato a ser solución en el espacio de búsqueda entonces se inicia $x$ en una posición aleatoria en el espacio de búsqueda y hasta que se cumplan un número de iteraciones o se cumpla un criterio de finalización, se muestrea una nueva posición $y$ de la hiperesfera de un radio dado que rodea la posición actual de $x$ y si $f(y) < f(x)$ entonces la nueva posición de $x$ es la de $y$ y cuando se cumple el criterio de terminación $x$ ocupa la mejor posición encontrada.

A diferencia de este algotitmo matemático básico con el que se inicio la búsqueda aleatoria, en el método utilizado para la optimización de hiperparámetros en la selección de modelos, la nueva posición $y$ de $x$ se muestrea agregando un vector aleatorio con distribución normal a la actual posición de $x$. 

A comparación de la búsqueda en cuadrícula, con la búsqueda aleatoria no se pierde la eficiencia al bajar la dimensionalidad cuando se tienen muchos parámetros. Por ejemplo, si tengo una función $f$ de dos variables y ésta se puede aproximar a una función con una sola variable ($f(x,y) \approx g(x)$), entonces la función $f$ tiene una baja dimensionalidad efectiva.

A continuación, se muestra la diferencia entre la búsqueda de cuadrícula y la aleatoria al optimizar la función $f(x, y) = g(x) + h(y) \approx g(x)$ con baja dimensionalidad efectiva, donde se observa como con la búsqueda aleatoria si se abarcan todos los distintos valores de $g$ para los parámetros.
<center><img src="GS_vs_RS.png" width="800"></center>

Nota. Adaptado de Grid and random search [Imagen], por Bergstra, J. & Bengio, Y., 2012, Journal of Machine Learning Research (https://www.jmlr.org/papers/volume13/bergstra12a/bergstra12a.pdf).

Matemáticamente, para un número dado de iteraciones $N$, en un conjunto $V$ de hiperparámetros. Se explora $N$ diferentes valores para cada hiperparámetro mientras en la búsqueda en cuadrícula se exploravan $N^{\frac{1}{V}}$ valores para cada hiperparámetro.

Con la librería RandomizedSearchCV  del paquete sklearn se puede implementar en Python la búsqueda aleatoria. Para ello, hay que escribir el siguiente comando

In [13]:
from sklearn.model_selection import RandomizedSearchCV

### Ejemplos

Definimos los posibles hiperparámetros del modelo y con la función RandomizedSearchCV encontramos cuales se ajustan mejor al modelo.

In [14]:
hyperparameters = dict(loss = ['squared_error', 'absolute_error', 'huber', 'quantile'],
                       learning_rate = [0.025, 0.05, 0.075, 0.1, 0.125, 0.15, 0.175, 0.2],
                       min_samples_split = [2, 3, 4, 5],
                       min_samples_leaf = [1, 2, 3, 4],
                       max_depth = [1, 2, 3, 4, 5, 6, 7],
                       max_features = ['sqrt', 'log2'],
                       n_estimators = [20, 40, 60, 80, 100, 120, 140, 160, 180, 200])

clf = RandomizedSearchCV(reg, hyperparameters, random_state=0)

X = data.iloc[:, :-1].values
y = data.iloc[:, -1].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, shuffle=True)

search = clf.fit(X_train, y_train)
parameters=search.best_params_
print('Mejores parámetros:', parameters)

Mejores parámetros: {'n_estimators': 180, 'min_samples_split': 4, 'min_samples_leaf': 3, 'max_features': 'sqrt', 'max_depth': 4, 'loss': 'huber', 'learning_rate': 0.2}


Posteriormente, ajustamos el modelo con esos hiperparámetros y calculamos el coeficiente de determinación R2.

In [15]:
reg = GradientBoostingRegressor(loss=parameters['loss'], 
                                learning_rate = parameters['learning_rate'], 
                                min_samples_split = parameters['min_samples_split'], 
                                min_samples_leaf = parameters['min_samples_leaf'], 
                                max_depth = parameters['max_depth'],
                                max_features = parameters['max_features'],
                                n_estimators = parameters['n_estimators'])
reg.fit(X_train, y_train)

y_pred = reg.predict(X_test)

r2 = r2_score(y_test, y_pred)
print('R2:', r2)

R2: 0.7952619140358745


Según el coeficiente de determinación se podría decir que para el modelo inicial los hiperparámetros escogidos por este método no son los mejores, dado que el coeficiente del modelo base es mayor.

## Optimización bayesiana

### Fundamento matemático

Con este método de optimización de hiperparámetros es posible reducir dramáticamente el costo computacional de encontrar el conjunto de hiperparámetros ideal para nuestro modelo dentro de nuestro espacio de búsqueda, y a la vez no lidia con el problema que presentan la búsqueda de grilla o la búsqueda aleatoria, que hace que el espacio de búsqueda sea mucho más reducido debido al costo computacional, y además nunca es garantizado que con la búsqueda aleatoria podremos obtener los hiperparámetros ideales.

En la optimización de hiperparámetros usando optimización bayesiana, se lleva un registro de todas las evaluaciones pasadas de posibles hiperparámetros, mediante un modelo probabilístico que mapea hiperparámetros a la probabilidad de tener cierto puntaje en la función objetivo dados dichos hiperparámetros. Este modelo probabilístico esta definido como

$$
p(score \ | \ hyperparameters)
$$

Este modelo es llamado en la literatura como un "sustituto" de la función objetivo que es la que queremos optimizar. Esta aproximación es un modelo probabilístico que dado un conjunto de hiperparámetros, se evalúa la función objetivo en este conjunto y se retorna un score que es usado para actualizar la distribución del modelo probabilístico, entonces con cada iteración, este modelo probabilístico se vuelve un predictor mas robusto de scores. 
La distribución se actualiza usando el teorema de Bayes, ya que se calcula de la siguiente forma:
$$
p(score \ | \ hyperparameters) = \frac{p(hyperparameters \ | \ score)\cdot p(score)}{p(hyperparameters)}
$$
donde
$$
p(hyperparameters \ | \ score) = 
\begin{cases}
l(x), \ if \text{ -score} < \text{ -score threshold} \\
g(x), \ if \text{ -score} \geq \text{ -score threshold} 
\end{cases}
$$
Cada vez que un conjunto de hiperparametros se evalúa en la función objetivo, se obtiene una mezcla gaussiana $l(x)$ ó $g(x)$ que depende del score obtenido. Este método es eficiente porque el conjunto de hiperparámetros que elige en cada iteración lo hace de una manera informada, guiandose por el modelo probabilístico.

Por lo anterior, en cada iteración el modelo sustituto se va ajustando más a la función objetivo, veamos que en una primera iteración, el modelo sustituto (linea negra continua) y la función objetivo (linea roja punteada) se ven así:

<center><img src="image-20220926-214500.png"></center>

Cada punto negro es una evaluación del modelo sustituto, y luego de 8 evaluaciones el modelo sustituto luce de la siguiente forma:

<center><img src="image-20220926-214733.png"></center>

Nota. Adaptado de A Conceptual Explanation of Bayesian Model Based Hyperparameter Optimization for Machine Learning (https://towardsdatascience.com/a-conceptual-explanation-of-bayesian-model-based-hyperparameter-optimization-for-machine-learning-b8172278050f)

Este modelo sustituto es también lo que se conoce como una distribución de probabilidad a priori, y el modelo que resulta despues de realizar varias evaluaciones en la función objetivo y formar el modelo sustituto, es lo que se conoce como probabilidad a posteriori sobre la función objetivo.

Hay distintas librerías para optimización bayesiana de hiperparámetros, a continuación mostramos la más utilizada y como se usa en Python

In [16]:
import optuna

/shared-libs/python3.9/py/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Ejemplos

A continuación vamos a dar un ejemplo donde usamos la librería optuna que es la más popular entre las demás.

Definimos la función objetivo, como esta función retorna el coeficiente de determinación R2 y entre más alto mejor, entonces esta función objetivo es una función que vamos a maximizar.

In [17]:
def objective(trial):
    # set hyperparameters
    _loss = trial.suggest_categorical('loss', {'squared_error', 'absolute_error', 'huber'})
    _learning_rate = trial.suggest_float('learning_rate', low = 0.025, high = 0.2, step = 0.025)
    _min_samples_split = trial.suggest_int('min_samples_split', low = 2, high = 5, step = 1)
    _min_samples_leaf = trial.suggest_int('min_samples_leaf', low = 1, high = 4, step = 1)
    _max_depth = trial.suggest_int('max_depth', low = 1, high = 7, step = 1)
    _max_features = trial.suggest_categorical('max_features', {'sqrt', 'log2'})
    _n_estimators = trial.suggest_int('n_estimators', low = 20, high = 200, step = 20)

    X = data.iloc[:, :-1].values
    y = data.iloc[:, -1].values

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, shuffle=True)

    reg = GradientBoostingRegressor(loss=_loss, 
                                    learning_rate = _learning_rate, 
                                    min_samples_split = _min_samples_split, 
                                    min_samples_leaf = _min_samples_leaf, 
                                    max_depth = _max_depth,
                                    max_features = _max_features,
                                    n_estimators = _n_estimators)
    reg.fit(X_train, y_train)

    y_pred = reg.predict(X_test)

    r2 = r2_score(y_test, y_pred)

    return r2

Creamos un objeto de tipo `optuna.Study` donde especificamos que estamos ante un problema de maximización, luego ejecutas el método `study.optimize` donde especificamos la función objetivo, el número de intentos como 100 y el número de cores a utilizar durante la optimización.

In [18]:
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100, n_jobs=-1)

[I 2022-11-28 01:49:04,493] A new study created in memory with name: no-name-37e52e3c-074c-448a-afc2-2e1d28d1da99
[I 2022-11-28 01:49:04,718] Trial 3 finished with value: 0.7980653551924303 and parameters: {'loss': 'squared_error', 'learning_rate': 0.07500000000000001, 'min_samples_split': 2, 'min_samples_leaf': 4, 'max_depth': 6, 'max_features': 'log2', 'n_estimators': 20}. Best is trial 3 with value: 0.7980653551924303.
[I 2022-11-28 01:49:05,140] Trial 4 finished with value: 0.7915604189213935 and parameters: {'loss': 'absolute_error', 'learning_rate': 0.2, 'min_samples_split': 3, 'min_samples_leaf': 3, 'max_depth': 2, 'max_features': 'sqrt', 'n_estimators': 20}. Best is trial 3 with value: 0.7980653551924303.
[I 2022-11-28 01:49:05,148] Trial 0 finished with value: 0.7493286815828024 and parameters: {'loss': 'squared_error', 'learning_rate': 0.07500000000000001, 'min_samples_split': 4, 'min_samples_leaf': 3, 'max_depth': 1, 'max_features': 'log2', 'n_estimators': 200}. Best is tria

Posterior a los intentos de maximizar el coeficiente de determinación, procedemos a mostrar el mejor conjunto de hiperparámetros y el coeficiente de determinación obtenido de este conjunto de hiperparámetros.

In [19]:
print('Mejor conjunto de hiperparametros:\n' + str(study.best_params))
print('Mejor coeficiente de determinación:', study.best_value)

Mejor conjunto de hiperparametros:
{'loss': 'absolute_error', 'learning_rate': 0.125, 'min_samples_split': 5, 'min_samples_leaf': 2, 'max_depth': 6, 'max_features': 'log2', 'n_estimators': 140}
Mejor coeficiente de determinación: 0.8902168656116525


Con el método en cuestión obtenemos un mejor coeficiente de determinación, entonces este método resulta ser efectivo.

## Ejercicio

El ejercicio propuesto para el lector es ajustar un árbol de decisión (DecisionTreeRegressor) de scikit-learn al conjunto de datos usado en esta exposición, y utilizar el método de optimización bayesiana para la optimización de hiperparámetros. El objetivo de este ejercicio es obtener un coeficiente de determinación (R2) mayor a 0.89 que fue el último obtenido con el método de optimización bayesiana.

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=006e66de-9f6a-4c40-bebc-c7a7dc0f0ecd' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>